In [4]:
# LEARNING: Importing LangGraph Core Components
# StateGraph: The main class for building stateful workflows in LangGraph
# START: Special constant representing the entry point of the graph
# END: Special constant representing the exit point of the graph
# TypedDict: Used to define the structure of state that flows through the graph
from langgraph.graph import StateGraph, START, END
from typing import TypedDict

In [7]:
# LEARNING: Defining State Schema with TypedDict
# In LangGraph, state is the data that flows through the graph nodes.
# TypedDict provides type hints for the state structure, making it clear what data is available.
# This BMIState will hold weight, height, and the calculated BMI value.
# Each node in the graph can read from and write to this state.
class BMIState(TypedDict):
    weight_kg: float
    height_m: float
    bmi: float

In [6]:
# LEARNING: Creating a Node Function
# In LangGraph, nodes are functions that process the state.
# Each node function:
#   1. Receives the current state as input
#   2. Performs some computation or logic
#   3. Returns the updated state
# This function calculates BMI using the formula: weight / (height^2)
# The state is modified in-place and then returned.
def calculate_bmi(state: BMIState) -> BMIState:
    weight= state['weight_kg']
    height = state['height_m']
    bmi= weight/(height**2)
    state['bmi'] = round(bmi, 2)
    return state

In [8]:
# LEARNING: Building and Configuring the Graph
# Step 1: Initialize StateGraph with the state schema (BMIState)
graph = StateGraph(BMIState)

# LEARNING: Adding Nodes to the Graph
# add_node() registers a function as a node in the graph.
# Parameters: node_name (string), node_function (callable)
graph.add_node('calculate_bmi', calculate_bmi)

# LEARNING: Defining Graph Flow with Edges
# Edges define the execution flow between nodes.
# add_edge(from_node, to_node) creates a directed connection.

# START is a special node representing the entry point of the workflow.
# This edge means: "When the workflow starts, go to the 'calculate_bmi' node"
graph.add_edge(START, 'calculate_bmi')

# END is a special node representing the exit point of the workflow.
# This edge means: "After 'calculate_bmi' completes, end the workflow"
graph.add_edge('calculate_bmi', END)

# LEARNING: Compiling the Graph into an Executable Workflow
# compile() converts the graph definition into a runnable workflow.
# After compilation, the workflow can be invoked with initial state.
# The compiled workflow validates the graph structure and optimizes execution.
workflow = graph.compile()

In [11]:
# LEARNING: Executing the Workflow
# Step 1: Create initial state with input data
# Note: We don't need to provide 'bmi' in the initial state - it will be calculated
initial_state = {'weight_kg':125, 'height_m':1.79}

# LEARNING: Invoking the Workflow
# invoke() executes the compiled workflow with the provided initial state.
# The workflow follows the defined edges: START -> calculate_bmi -> END
# It returns the final state after all nodes have been executed.
final_state = workflow.invoke(initial_state)

# The final state now includes the calculated BMI value (39.01)
# BMI = 125 / (1.79^2) = 39.01
print(final_state)

{'weight_kg': 125, 'height_m': 1.79, 'bmi': 39.01}
